In [1]:
import os
from dfply import *
import pandas as pd
import datetime
np.set_printoptions(suppress=True) 
os.chdir(r"D:\03lecture\QT\codeBackup\data")
from math import *

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def end_of_month(any_day):
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)

# formatting of msf
msf = pd.read_csv("msf.csv")
msf["date"] = pd.to_datetime(msf["date"], format='%Y%m%d')
msf["date"] = [end_of_month(date) for date in msf["date"]]
msf.columns = [i.lower() for i in msf.columns]
msf["month"] = [date.month for date in msf["date"]]
msf["year"] = msf.date.apply(lambda x:x.year)
# formatting of mse

In [4]:
mse = pd.read_csv("mse.csv")

In [5]:
mse.columns = [i.lower() for i in mse.columns]

In [6]:
mse["exdt"] = pd.to_datetime(mse["exdt"], format='%Y%m%d')
mse["dclrdt"] = pd.to_datetime(mse["dclrdt"], format='%Y%m%d')
mse["paydt"] = pd.to_datetime(mse["paydt"], format='%Y%m%d')
mse = mse >> mask(X.divamt == X.divamt) ## dropna subset == divamt

In [36]:
mse['date'] = mse['date'] = mse.exdt.agg(end_of_month)
# merge mse and msf

In [37]:
mse['freq'] = mse.distcd.apply(lambda x:str(x)[:3])
mse['freq'] = mse.freq.astype('int64')
mse['freq'].loc[mse['freq'] <= 123] = 123 ## 012全部作为quarterly data 对待

mseUse = pd.DataFrame(mse.groupby(['permno','date','distcd','freq'])['divamt'].sum()).reset_index()

In [299]:
data = pd.merge(msf,mseUse,how = 'left',on = ['permno','date'])

In [300]:
data.pop('bidlo')
data.pop('askhi')
data["mcap"] = data["prc"] * data["shrout"]
# fill na with distcd if the stock has distcd before so that we can identify stock that pay dividend
data = data >> arrange(X.permno,X.date)
data['freq'] = data.groupby('permno')['freq'].fillna(method = 'ffill')

In [301]:
data['distcd'] = data.groupby('permno')['distcd'].fillna(method = 'ffill')

In [302]:
divDataSource = data.drop_duplicates(subset = ['permno','date']) ## 如果同一个月即有123 又有非123的 保留，

In [303]:
## creat lag
divDataSource.sort_values(['permno','date'],inplace = True)
divDataSource['prc_lag1'] = divDataSource.groupby('permno')['prc'].shift(1)
divDataSource['mcap_lag1'] = divDataSource.groupby('permno')['mcap'].shift(1)
divDataSource['freq_lag1'] = divDataSource.groupby('permno')['freq'].shift(1)
divDataSource['turnover'] = divDataSource['vol']/divDataSource['shrout']
divDataSource['spread_2'] = divDataSource['ask'] -divDataSource['bid']

In [304]:
divDataSource['Div_Yield'] = divDataSource['divamt']/divDataSource['prc']

In [305]:
for i in range(12):
    div_lag = 'div_lag' + str(i + 1)
    divDataSource[div_lag] = divDataSource.groupby('permno')['divamt'].shift(i+1)

In [306]:
import feather

In [307]:
linkTable = feather.read_dataframe('monthly_gvkey_permno_link.feather')
linkTable['date'] = pd.to_datetime(linkTable['date'])

linkTable['N_permno']=linkTable.groupby('permno')['permno'].transform('count')
linkTable['year'] = linkTable.date.apply(lambda x:x.year)

linkTable.drop_duplicates(subset = ['gvkey','permno','year'],inplace = True)

linkTable['N_permno'] = linkTable.groupby(['year','gvkey'])['permno'].transform('count')

linkTable.query('N_permno >=2').sort_values(by = ['gvkey','year'])

BMdata = pd.read_csv('BMdata.csv')
BMdata['datadate'] = pd.to_datetime(BMdata.datadate,format = '%Y%m%d')
BMdata = BMdata[['gvkey','datadate','bkvlps']]

BMdata.rename(columns= {'datadate':'date'},inplace = True)
BMdata['year'] = BMdata.date.apply(lambda x:x.year)

BMdata_permno = BMdata.merge(linkTable,on = ['gvkey','year'],how= 'left',suffixes = ['_BM','_link'])
BMdata_permno.dropna(subset = ['bkvlps'],inplace= True)
BMdata_permno['permno'] = BMdata_permno.groupby('gvkey')['permno'].fillna(method = 'ffill')
BMdata_permno.drop_duplicates(subset = ['permno','date_BM','bkvlps'],inplace = True)
BMdata_permno.dropna(subset = ['permno'],inplace = True)
BMdata_permno['N_permno'] = BMdata_permno.groupby(['permno','date_BM'])['permno'].transform('count')
BMuse = pd.DataFrame(BMdata_permno.groupby(['permno','date_BM'])['bkvlps'].mean()).reset_index()
BMdata_permno.rename(columns = {'date_BM':'date'},inplace = True)

In [308]:
BMdata_permno.head()

,gvkey,date,bkvlps,year,permno,date_link,N_permno
9,1000,1970-12-31,4.3107,1970,25881.0,1970-11-30,1
10,1000,1971-12-31,2.7987,1971,25881.0,1971-01-29,1
11,1000,1972-12-31,2.4194,1972,25881.0,1972-01-31,1
12,1000,1973-12-31,3.0165,1973,25881.0,1973-01-31,1
13,1000,1974-12-31,4.5781,1974,25881.0,1974-01-31,1


In [309]:
divDataSourceBM = pd.merge(divDataSource,BMdata_permno.drop(columns = ['year']),on = ['permno','date'],how = 'left')
divDataSourceBM = divDataSourceBM >> arrange(X.permno,X.date)
divDataSourceBM['bkvlps'] = divDataSourceBM.groupby('permno')['bkvlps'].fillna(method = 'ffill')
divDataSourceBM['bm'] = divDataSourceBM['bkvlps']/divDataSource['prc']

In [310]:
dataUse = divDataSourceBM.query('date <= "2011-12-31" &'
                          'shrcd in([10,11]) &'
                          'hexcd in([1,2,3]) &'
                          'prc_lag1 >= 5 &'
                          'ret not in(["B","C"])')

In [311]:
dataUse['ret'] = dataUse.ret.astype('float')

In [312]:
divData = dataUse.query('freq in(["120","121","123","124","125"])')

### 1 creat table1

#### 1.1  creat table1PanelA

In [313]:
table1PanelApart1 = divData[['mcap','turnover','spread_2','Div_Yield']].describe().T

In [314]:
table1PanelApart2 = divData.query('bm <500 & bm >0')[['bm']].describe().T

In [315]:
table1PanelA = pd.concat([table1PanelApart1,table1PanelApart2])

In [316]:
N_firms = pd.DataFrame({'count':len(set(divData['permno']))},index = ['N_firms'])

In [317]:
table1PanelA = pd.concat([table1PanelA,N_firms])

In [318]:
table1PanelA

,25%,50%,75%,count,max,mean,min,std
mcap,37168.812500,149425.000000,674204.625000,1312290.0,5.810989e+08,1.864001e+06,-1.300461e+08,1.077782e+07
turnover,0.129215,0.290577,0.652810,1316192.0,2.324402e+02,5.868285e-01,0.000000e+00,1.110118e+00
spread_2,0.080000,0.250000,0.500000,597116.0,1.724800e+02,4.261916e-01,-3.500000e+00,8.598707e-01
Div_Yield,0.004167,0.007692,0.012085,376759.0,2.727273e+00,8.342993e-03,-4.444444e-01,1.157757e-02
bm,0.415026,0.865341,1.966747,780765.0,4.875328e+02,2.566743e+00,6.909091e-05,9.163280e+00
N_firms,NaN,NaN,NaN,8326.0,NaN,NaN,NaN,NaN


#### 1.2 Creat table1PanelB

In [319]:
NoDivData = dataUse.query('freq not in(["120","121","123","124","125"])')

In [320]:
table1PanelBpart1 = NoDivData[['mcap','turnover','spread_2']].describe().T
table1PanelBpart2 = NoDivData.query('bm <1000&bm >0')[['bm']].describe().T

In [321]:
table1PanelBpart1

,count,mean,std,min,25%,50%,75%,max
mcap,693234.0,908446.377600,6.180054e+06,-1.776986e+07,43993.218750,140175.000000,464997.367500,6.024329e+08
turnover,696685.0,1.545840,2.397334e+00,0.000000e+00,0.366183,0.885089,1.915989,3.041669e+02
spread_2,558218.0,0.369774,4.313983e+00,-3.312500e+00,0.050000,0.200000,0.375000,6.500000e+02


In [322]:
N_firms2 = pd.DataFrame({'count':len(set(NoDivData.permno))},index = ['N_firms'])

In [323]:
N_firms2

,count
N_firms,15917


In [324]:
table1PanelB= pd.concat([table1PanelBpart1,table1PanelBpart2,N_firms2])

In [325]:
table1PanelB

,25%,50%,75%,count,max,mean,min,std
mcap,43993.218750,140175.000000,464997.367500,693234.0,6.024329e+08,908446.377600,-1.776986e+07,6.180054e+06
turnover,0.366183,0.885089,1.915989,696685.0,3.041669e+02,1.545840,0.000000e+00,2.397334e+00
spread_2,0.050000,0.200000,0.375000,558218.0,6.500000e+02,0.369774,-3.312500e+00,4.313983e+00
bm,0.179030,0.433709,1.127327,475330.0,9.507427e+02,1.756866,1.845505e-05,1.133169e+01
N_firms,NaN,NaN,NaN,15917.0,NaN,NaN,NaN,NaN


### <font face = 'Times New Roman'> 1.3 Panel C column 1: Percent of firm-months with dividend in the last year

In [326]:
dataUse.columns

Index(['permno', 'date', 'shrcd', 'hexcd', 'prc', 'vol', 'ret', 'bid', 'ask',
       'shrout', 'spread', 'retx', 'month', 'year', 'distcd', 'freq', 'divamt',
       'mcap', 'prc_lag1', 'mcap_lag1', 'freq_lag1', 'turnover', 'spread_2',
       'Div_Yield', 'div_lag1', 'div_lag2', 'div_lag3', 'div_lag4', 'div_lag5',
       'div_lag6', 'div_lag7', 'div_lag8', 'div_lag9', 'div_lag10',
       'div_lag11', 'div_lag12', 'gvkey', 'bkvlps', 'date_link', 'N_permno',
       'bm'],
      dtype='object')

In [328]:
dataUse['divType'] = dataUse.distcd.apply(lambda x:str(x)[:3])

In [329]:
DivFreq = pd.DataFrame(dataUse.groupby(['year','divType'])['divType'].count())
DivFreq['percent'] = DivFreq.divType/DivFreq.groupby(level = 0).divType.transform(np.sum)
DivFreq.columns = ['count','percent']
DivFreq = DivFreq.reset_index()

In [296]:
DivFreq

,year,divType,count,percent
0,1927,121,39,0.008752
1,1927,122,59,0.013241
2,1927,123,2510,0.563285
3,1927,124,66,0.014811
4,1927,127,15,0.003366
5,1927,133,4,0.000898
6,1927,181,1,0.000224
7,1927,187,6,0.001346
8,1927,nan,1756,0.394075
9,1928,121,78,0.014208


In [291]:
dataUse['divType'] = dataUse.groupby('permno')['divType'].fillna(method = 'ffill')

In [294]:
dataUse[['permno','divType','freq']]

,permno,divType,freq
29,10001,nan,123.0
30,10001,nan,123.0
31,10001,123,123.0
32,10001,123,123.0
33,10001,123,123.0
34,10001,123,123.0
35,10001,123,123.0
38,10001,123,123.0
39,10001,123,123.0
40,10001,123,123.0


In [398]:
DivFreq= pd.DataFrame(DivFreq.groupby('divType')['percent'].mean())

In [395]:
div_freq = ['120','122','123','124','125','126','127']
Any_freq = table1PanelC_part1.loc[div_freq,].sum()
div_freq_index = {'monthly':'122','quarterly':'123','semiAnnualy':'124','Annualy':'125'}

In [432]:
DivFreq.head(10)

,percent
divType,
120,6.60634e-05
121,0.162015
122,0.00352384
123,0.562652
124,0.016181
125,0.0100621
126,0.00474289
127,0.0281474
128,0.0024635


In [434]:
table1PanelC_part1 = DivFreq.loc[['122','123','124','125','121'],]

In [435]:
table1PanelC_part1

,percent
divType,
122,0.00352384
123,0.562652
124,0.016181
125,0.0100621
121,0.162015


In [436]:
table1PanelC_part1.loc['Any_freq']= {'percent':Any_freq.tolist()[0]}

In [437]:
table1PanelC_part1

,percent
divType,
122,0.00352384
123,0.562652
124,0.016181
125,0.0100621
121,0.162015
Any_freq,0.625375


In [412]:
table1PanelC_part1.rename(index= {"122":'monthly',"123":"quarterly","124":"semiAnnualy","125":"Annualy"},inplace = True)

In [413]:
table1PanelC_part1

,percent
divType,
monthly,0.00352384
quarterly,0.562652
semiAnnualy,0.016181
Annualy,0.0100621
Any_freq,0.625375


### <font face = 'Times New Roman'>  Panel C-Column 2: Percent of dividend observations

In [418]:
divData['divType'] = divData.distcd.apply(lambda x:str(x)[:3])

In [419]:
DivFreq2 = pd.DataFrame(divData.groupby('divType')['divType'].count())

In [420]:
DivFreq2.columns = ['count']

In [421]:
DivFreq2

,count
divType,
120,5
121,133836
122,3313
123,1118490
124,37516
125,24031


In [422]:
DivFreq2['percent'] = DivFreq2['count']/DivFreq2['count'].sum()

In [429]:
table1PanelC_part2 = DivFreq2[['percent']].loc[['122','123','124','125','121'],]

In [448]:
table1PanelC = pd.merge(table1PanelC_part1,table1PanelC_part2,left_index = True,right_index = True,how = 'outer')

In [450]:
table1PanelC.rename(index = {'121':'unknow_freq','122':'monthly','123':'quarterly','124':'semiAnnualy','125':'Annualy'},inplace = True)

In [454]:
table1PanelC = round(table1PanelC.apply(lambda x:x*100),2)

In [455]:
table1PanelC

,percent_x,percent_y
divType,,
unknow_freq,16.2015,10.16
monthly,0.352384,0.25
quarterly,56.2652,84.91
semiAnnualy,1.6181,2.85
Annualy,1.00621,1.82
Any_freq,62.5375,NaN


### 2  creat Table2

#### 2.1 creat Table2 PanelA

In [218]:
table2_mean = {}
table2_std = {}
table2_all_div_prob = {}
table2_quar_div_prob = {}

In [210]:
divData.columns.tolist().index('div_lag1')

23

In [211]:
divData.columns.tolist().index('div_lag12')

34

In [214]:
div_lag_list = divData.columns[23:35].tolist()

In [215]:
div_lag_list

['div_lag1',
 'div_lag2',
 'div_lag3',
 'div_lag4',
 'div_lag5',
 'div_lag6',
 'div_lag7',
 'div_lag8',
 'div_lag9',
 'div_lag10',
 'div_lag11',
 'div_lag12']

In [219]:
for i,div_lag in enumerate(div_lag_list):
    table2_mean[str(i + 1)] = divData.dropna(subset = [div_lag])['ret'].mean()*100
    table2_std[str(i + 1)] = divData.dropna(subset = [div_lag] )['ret'].std()*100
    table2_all_div_prob[str(i+1)] = len(divData.dropna(subset = [div_lag,'divamt']))/len(divData.dropna(subset = [div_lag]))
    divData_quar = divData.query('freq == "123"')
    table2_quar_div_prob[str(i + 1)] = len(divData_quar.dropna(subset = [div_lag,'divamt']))/len(divData_quar.dropna(subset = [div_lag]))

In [220]:
table2_panelA = pd.DataFrame(pd.Series(table2_mean),columns = ['Mean return'])
table2_panelA['Standard Deviation'] = table2_std.values()
table2_panelA['All dividends'] = table2_all_div_prob.values()
table2_panelA['quar dividends'] = table2_quar_div_prob.values()

In [222]:
table2_panelA#  ## quartly dividends 的return 有些偏高，因该是120，121，122都归类为123的原因，后面可以再调整下。

,Mean return,Standard Deviation,All dividends,quar dividends
1,1.047115,9.721843,0.010833,0.010986
2,1.191799,9.838917,0.062552,0.063514
3,1.372169,9.599297,0.858883,0.872877
4,1.007758,9.745201,0.064253,0.065057
5,1.197198,9.882160,0.068914,0.068576
6,1.389761,9.639035,0.855029,0.860230
7,1.031427,9.786326,0.069624,0.069264
8,1.171970,9.915469,0.067114,0.067989
9,1.367257,9.629136,0.840041,0.853276
10,1.007349,9.823710,0.067378,0.068255


#### 2.2 creat table2 PanelB

In [196]:
# assign signal
divData["signal"] = np.where((divData.div_lag3.notna() | divData.div_lag6.notna() |
                                 divData.div_lag9.notna() | divData.div_lag12.notna()), "L",
                                np.where(divData.div_lag1.notna() | divData.div_lag2.notna() |
                                         divData.div_lag4.notna() | divData.div_lag5.notna() |
                                         divData.div_lag7.notna() | divData.div_lag8.notna() |
                                         divData.div_lag10.notna() | divData.div_lag11.notna(), "S", ""))

portfolio1 = pd.DataFrame(divData.groupby(["date", "signal"])['ret'].mean())

In [229]:
probs = [0.01,0.05,0.25,0.5,0.75,0.95,0.99]

In [257]:
table2PanelB_part1 = portfolio1.groupby('signal')['ret'].describe(percentiles = probs).loc[['L','S']].drop(
    columns = ['count','max','min']).apply(lambda x:x*100)

In [258]:
table2PanelB_part1

,mean,std,1%,5%,25%,50%,75%,95%,99%
signal,,,,,,,,,
L,1.373574,5.928246,-16.403156,-7.681396,-1.438040,1.657452,4.384842,9.217188,16.602504
S,1.067091,6.066403,-17.101328,-8.112217,-1.658579,1.485880,4.068795,8.861298,15.205930


In [272]:
table2PanelB_part2 = pd.DataFrame(NoDivData.groupby('date')['ret'].mean()).describe(percentiles = probs).T.drop(columns = ['count','min','max']).apply(lambda x:x*100)

In [273]:
table2PanelB = pd.concat([table2PanelB_part1,table2PanelB_part2])

In [279]:
table2PanelB['index'] = ['div_predM','div_NoPredM','NoDiv']

In [282]:
table2PanelB.set_index('index',inplace = True)

In [283]:
table2PanelB

,mean,std,1%,5%,25%,50%,75%,95%,99%
index,,,,,,,,,
div_predM,1.373574,5.928246,-16.403156,-7.681396,-1.438040,1.657452,4.384842,9.217188,16.602504
div_NoPredM,1.067091,6.066403,-17.101328,-8.112217,-1.658579,1.485880,4.068795,8.861298,15.205930
NoDiv,0.980859,8.370443,-23.399086,-11.848510,-3.330714,1.536198,5.426408,13.156111,21.093995
